In [0]:
%pip install snowflake-snowpark-python

  Using cached snowflake_snowpark_python-1.44.0-py3-none-any.whl.metadata (16 kB)
  Using cached snowflake_connector_python-4.1.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (78 kB)
  Using cached tzlocal-5.3.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached asn1crypto-1.5.1-py2.py3-none-any.whl.metadata (13 kB)
  Using cached cryptography-46.0.3-cp311-abi3-manylinux_2_34_x86_64.whl.metadata (5.7 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tomlkit-0.13.3-py3-none-any.whl.metadata (2.8 kB)
  Using cached cffi-2.0.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (2.6 kB)
INFO: pip is looking at multiple versions of pyopenssl to determine which version is compatible with other requirements. This could take a while.
  Using cached pyopenssl-25.3.0-py3-none-any.whl.metadata (17 kB)
Using cached snowflake_snowpark_python-1.44.0-py3-none-any.whl (1.8 MB)
Using cached snowflake_connector_python-4.1.1-cp312-

In [0]:
storage_account = "snowflakehexa"
access_key = "s+dpBFiy973QoHY+hvsWQHQOtSglbQMOLcSPJzr6TT35DnKIzpdHgDuXHg7Ss9maUparnjBtbxrt+AStvpvY3A=="

spark.conf.set(
  f"fs.azure.account.key.snowflakehexa.blob.core.windows.net",
  "s+dpBFiy973QoHY+hvsWQHQOtSglbQMOLcSPJzr6TT35DnKIzpdHgDuXHg7Ss9maUparnjBtbxrt+AStvpvY3A=="
)

In [0]:
customers_df = spark.read.option("header", "true").csv(
    "wasbs://data@snowflakehexa.blob.core.windows.net/customers (1).csv"
)

orders_df = spark.read.option("header", "true").csv(
    "wasbs://data@snowflakehexa.blob.core.windows.net/orders.csv"
)

payments_df = spark.read.option("header", "true").csv(
    "wasbs://data@snowflakehexa.blob.core.windows.net/payments_2024_02.csv"
)
customers_df.show()
orders_df.show()
payments_df.show()


+-----------+----------+---------+---------------+---------+------------+
|customer_id|first_name|last_name|          email|     city|created_date|
+-----------+----------+---------+---------------+---------+------------+
|        101|     Ankur|   Sharma|ankur@gmail.com|Bangalore|  2024-01-10|
|        102|      Riya|    Mehta| riya@gmail.com|   Mumbai|  2024-01-15|
|        103|     Karan|    Singh|karan@gmail.com|    Delhi|  2024-02-01|
+-----------+----------+---------+---------------+---------+------------+

+--------+-----------+----------+------------+------------+
|order_id|customer_id|order_date|order_amount|order_status|
+--------+-----------+----------+------------+------------+
|    5001|        101|2024-02-10|        1500|   COMPLETED|
|    5002|        102|2024-02-11|        2200|   COMPLETED|
|    5003|        101|2024-02-15|        1800|   CANCELLED|
+--------+-----------+----------+------------+------------+

+----------+--------+------------+------------+-------------

In [0]:
sfOptions = {
  "sfURL": "QYQSSEQ-UT92541.snowflakecomputing.com",
  "sfDatabase": "RETAIL_DB",
  "sfSchema": "RAW",
  "sfWarehouse": "RETAIL_WH",
  "sfRole": "ANALYTICS_ROLE",
  "sfUser": "BARATH46",
  "sfPassword": "Rbarathraj@2711"
}

customers_df.write.format("snowflake") \
  .options(**sfOptions) \
  .option("dbtable", "RAW_CUSTOMERS") \
  .mode("overwrite") \
  .save()

orders_df.write.format("snowflake") \
  .options(**sfOptions) \
  .option("dbtable", "RAW_ORDERS") \
  .mode("overwrite") \
  .save()

payments_df.write.format("snowflake") \
  .options(**sfOptions) \
  .option("dbtable", "RAW_PAYMENTS") \
  .mode("overwrite") \
  .save()

In [0]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, concat_ws, count, sum, max, lit

session = Session.builder.configs({
    "account": "QYQSSEQ-UT92541",
    "user": "BARATH46",
    "password": "Rbarathraj@2711",
    "role": "ANALYTICS_ROLE",
    "warehouse": "RETAIL_WH",
    "database": "RETAIL_DB",
    "schema": "RAW"
}).create()

customers = session.table("RAW_CUSTOMERS")
orders = session.table("RAW_ORDERS").filter(col("ORDER_STATUS") == "COMPLETED")
payments = session.table("RAW_PAYMENTS")

customers_curated = customers.select(
    col("CUSTOMER_ID"),
    concat_ws(
        lit(" "),               
        col("FIRST_NAME"),
        col("LAST_NAME")
    ).alias("FULL_NAME"),
    col("EMAIL"),
    col("CITY"),
    col("CREATED_DATE").alias("CUSTOMER_SINCE")
)

customers_curated.write.mode("overwrite").save_as_table("CURATED.CUSTOMERS_CURATED")
orders.write.mode("overwrite").save_as_table("CURATED.ORDERS_CURATED")
payments.write.mode("overwrite").save_as_table("CURATED.PAYMENTS_CURATED")

In [0]:
joined = customers_curated \
    .join(orders, "CUSTOMER_ID") \
    .join(payments, "ORDER_ID")

summary = joined.group_by(
    "CUSTOMER_ID", "FULL_NAME"
).agg(
    count("ORDER_ID").alias("TOTAL_ORDERS"),
    sum("ORDER_AMOUNT").alias("TOTAL_SALES"),
    max("ORDER_DATE").alias("LAST_ORDER_DATE")
)

summary.write.mode("overwrite").save_as_table(
    "ANALYTICS.CUSTOMER_SALES_SUMMARY"
)